In [6]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [7]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [8]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes

        #Inicialização dos pesos de forma mais adequada para o ReLu
        # self.biases = [np.random.randn(y, 1) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])] # * 0.5 foi add como ajuste para taxas maiores
        # self.weights = [np.random.randn(y, x) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])]

        #Inicialização dos pesos padrão
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def batch_normalize(self, x):
      mean = np.mean(x, axis=0)
      std = np.std(x, axis=0)
      return (x - mean) / (std + 1e-8)

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.batch_normalize(x)
            x = sigmoid(np.dot(w, x)+b) #net = (∑xw+b)
            #x = relu(np.dot(w, x)+b) #net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)

        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)

            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]


        #SUGESTAO CLOUDE SONNET
        max_grand_norm = 1.0
        for i in range(len(nabla_w)):
          grad_norm = np.linalg.norm(nabla_w[i])

          if (grad_norm > max_grand_norm):
            nabla_w[i] = nabla_w[i] * (max_grand_norm/grad_norm)

          #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
          self.weights[i] -= (η / len(mini_batch)) * nabla_w[i]
          self.biases[i] -= (η / len(mini_batch)) * nabla_b[i]



        # #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        # self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        # self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x]

        # Lista para armazenar todos os vetores net, camada por camada
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b
            nets.append(net)
            activation = sigmoid(net) #z = valor de saída do neurônio
            #activation = relu(net) #z = valor de saída do neurônio
            activations.append(activation)

        # Backward pass

        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1]) #SIGMOID
        #delta = self.cost_derivative(activations[-1], y) * relu_prime(nets[-1]) #RELU
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante.
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = sigmoid_prime(net)
            #zs = relu_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Função de Ativação ReLu
def relu(net):
    return np.maximum(0, net)

# Função para retornar as derivadas da função ReLu
def relu_prime(z):
    return np.where(z <= 0, 0, 1)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [9]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [ ]:
# Modificar o código para tratar corretamente as arquiteturas

learning_rate = [0.1, 0.3, 0.5]

one_layer_arquitecture = [
  [784, 10, 10],
  [784, 20, 10],
  [784, 30, 10]
]

two_layer_arquitecture = [
  [784, 20, 15, 10],
  [784, 30, 20, 10],
  [784, 40, 25, 10]
]

architectures = [one_layer_arquitecture, two_layer_arquitecture]

for arq_grupo in architectures:
    print(f"Executando com grupo das seguintes configurações: {arq_grupo}\n")
    for arq in arq_grupo:
        print(f"Configuração especifica a ser testada: {arq}")
        print("----------------------------------------------------")
        for x in learning_rate:
            training_data, validation_data, test_data = load_data_wrapper()
            training_data = list(training_data)

            # Criar a rede com a arquitetura específica
            mlp = Network(arq)
            print(f"Executando com taxa de aprendizado de {x}")
            mlp.SGD(training_data, 10, 32, x, test_data=test_data)
            print("\n")

print("Fim dos testes!")

Executando com grupo das seguintes configurações: [[784, 10, 10], [784, 20, 10], [784, 30, 10]]

Configuração especifica a ser testada: [784, 10, 10]
----------------------------------------------------
Executando com taxa de aprendizado de 0.1
Epoch 0 : 1602 / 10000 = 16.02%
Epoch 1 : 2319 / 10000 = 23.19%
Epoch 2 : 2635 / 10000 = 26.35%
Epoch 3 : 3058 / 10000 = 30.58%
Epoch 4 : 3423 / 10000 = 34.23%
Epoch 5 : 3768 / 10000 = 37.68%
Epoch 6 : 4016 / 10000 = 40.16%
Epoch 7 : 4182 / 10000 = 41.82%
Epoch 8 : 4464 / 10000 = 44.64%
Epoch 9 : 4733 / 10000 = 47.33%


Executando com taxa de aprendizado de 0.3
Epoch 0 : 2347 / 10000 = 23.47%
Epoch 1 : 3573 / 10000 = 35.73%
Epoch 2 : 4387 / 10000 = 43.87%
Epoch 3 : 4911 / 10000 = 49.11%
Epoch 4 : 5453 / 10000 = 54.53%
Epoch 5 : 5799 / 10000 = 57.99%
Epoch 6 : 6097 / 10000 = 60.97%
Epoch 7 : 6274 / 10000 = 62.74%
Epoch 8 : 6500 / 10000 = 65.0%
Epoch 9 : 6669 / 10000 = 66.69%


Executando com taxa de aprendizado de 0.5
Epoch 0 : 3764 / 10000 = 37.

##**Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.1, 0.3 e 0.5
     b) Função de ativação RELU
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline
Sigmoide  & 34,10& 31.24& 33.07& 52.01&  54.77& 50.74& 65.85&  65.29&  54.45& \\ \hline
Relu  & 71.67& 73.23& 74.41&  58.83&  80.28& 74.61& 49.74& 71.33 & 83.48 & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline
Sigmoide  & 36.59& 36.80& 38.35& 56.12 & 38.56 & 40.05& 63.98& 61.44 & 57.65 & \\ \hline
Relu  & 71.74& 72.36& 53.67& 58.41 & 81.56 & 59.98& 61.79& 69.23 & 81.84 & \\ \hline
\end{array}

2) Modifique a taxa de aprendizagem pra diminuir com o tempo. Execute com a melhor configuração encontrada. Melhorou o resultado?
